In [23]:
import json 
import pandas as pd
import numpy as np
import time
from scipy.sparse import dok_matrix
import itertools
from sklearn import preprocessing

In [83]:
data = json.load(open('./mpd.v1/data/mpd.slice.0-999.json'))

n_listas = 70

In [84]:
lists_data = []

for i in range(0,n_listas):
    for x in data['playlists'][i]['tracks']:
        lists_data.append([data['playlists'][i]['name'],data['playlists'][i]['num_albums'], data['playlists'][i]['num_artists'],data['playlists'][i]['num_followers'],data['playlists'][i]['num_tracks'], data['playlists'][i]['pid'], x['pos'], x['artist_name'], x['track_name'], x['duration_ms'], x['album_name']])

df = pd.DataFrame(lists_data, columns = ['play_list_name', 'num_albums', 'num_artists', 'num_followers', 'num_tracks', 'pid', 'song_pos', 'song_artist', 'song_name', 'song_duration', 'song_album'])

In [85]:
df.head()

,play_list_name,num_albums,num_artists,num_followers,num_tracks,pid,song_pos,song_artist,song_name,song_duration,song_album
0,Throwbacks,47,37,1,52,0,0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook
1,Throwbacks,47,37,1,52,0,1,Britney Spears,Toxic,198800,In The Zone
2,Throwbacks,47,37,1,52,0,2,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,47,37,1,52,0,3,Justin Timberlake,Rock Your Body,267266,Justified
4,Throwbacks,47,37,1,52,0,4,Shaggy,It Wasn't Me,227600,Hot Shot


In [86]:
# listas de listas de canciones 
listas = {}
cont = 0 

for i in range(0,n_listas):
    spot = []
    for x in data['playlists'][i]['tracks']:
        cont+=1
        spot.append(x['track_name'])

        if cont == len(data['playlists'][i]['tracks']):
            listas[data['playlists'][i]['pid']] = spot       
            cont = 0 
            spot = []


In [87]:
print(len(df.pid.unique()))
print(len(df.song_name.unique()))

70
3526


In [88]:
inicio = time.time()

songs_unique = df.song_name.unique()

#le = preprocessing.LabelEncoder()
#enc_songs = le.fit(songs_unique)
#size = len(le.classes_)
#matriz = dok_matrix((size,size))

pid_unique = df.pid.unique()

ejes = []

for (i,j) in itertools.combinations(songs_unique,2):
    for k in pid_unique:
        if j in listas[k] and i in listas[k] and i!=j:
            ejes.append([i,j])
               
fin = time.time()

print('tiempo demorado: {} segs'.format(fin - inicio))

tiempo demorado: 523.9549491405487 segs


In [79]:
df_result = pd.DataFrame(ejes, columns = ['Target', 'Source'])

df_result.to_csv('spotify_song_graph.csv', sep=';', index=False)

In [82]:
graph_labels = []

for x in songs_unique:
    graph_labels.append([x,x])

df_labels = pd.DataFrame(graph_labels, columns = ['id', 'label'])

df_labels.to_csv('song_graph_labels.csv', sep=';', index=False)
